In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [14, 15, 17]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 2.17890465259552, Final Batch Loss: 1.0856107473373413
Epoch 2, Loss: 2.1756207942962646, Final Batch Loss: 1.0881683826446533
Epoch 3, Loss: 2.172484040260315, Final Batch Loss: 1.0921766757965088
Epoch 4, Loss: 2.163689136505127, Final Batch Loss: 1.0777283906936646
Epoch 5, Loss: 2.1630829572677612, Final Batch Loss: 1.0812480449676514
Epoch 6, Loss: 2.1561315059661865, Final Batch Loss: 1.0729554891586304
Epoch 7, Loss: 2.141640067100525, Final Batch Loss: 1.0662823915481567
Epoch 8, Loss: 2.1359835863113403, Final Batch Loss: 1.069787621498108
Epoch 9, Loss: 2.123955726623535, Final Batch Loss: 1.059021234512329
Epoch 10, Loss: 2.116565465927124, Final Batch Loss: 1.0589808225631714
Epoch 11, Loss: 2.1026781797409058, Final Batch Loss: 1.0537225008010864
Epoch 12, Loss: 2.0900216102600098, Final Batch Loss: 1.0458992719650269
Epoch 13, Loss: 2.0687891244888306, Final Batch Loss: 1.0298360586166382
Epoch 14, Loss: 2.052584171295166, Final Batch Loss: 1.02733111381530

Epoch 124, Loss: 0.0841636024415493, Final Batch Loss: 0.04312380030751228
Epoch 125, Loss: 0.13602055236697197, Final Batch Loss: 0.05359899625182152
Epoch 126, Loss: 0.12838851287961006, Final Batch Loss: 0.056383755058050156
Epoch 127, Loss: 0.10134702920913696, Final Batch Loss: 0.04733125865459442
Epoch 128, Loss: 0.08835840225219727, Final Batch Loss: 0.03603600338101387
Epoch 129, Loss: 0.089250098913908, Final Batch Loss: 0.026749413460493088
Epoch 130, Loss: 0.0990169532597065, Final Batch Loss: 0.045787613838911057
Epoch 131, Loss: 0.09214109554886818, Final Batch Loss: 0.039983101189136505
Epoch 132, Loss: 0.09923917055130005, Final Batch Loss: 0.05149802193045616
Epoch 133, Loss: 0.09697598218917847, Final Batch Loss: 0.0622529499232769
Epoch 134, Loss: 0.1107722707092762, Final Batch Loss: 0.07532406598329544
Epoch 135, Loss: 0.10068311914801598, Final Batch Loss: 0.037699680775403976
Epoch 136, Loss: 0.11499547213315964, Final Batch Loss: 0.08755038678646088
Epoch 137, Lo

Epoch 247, Loss: 0.01824135333299637, Final Batch Loss: 0.015533833764493465
Epoch 248, Loss: 0.022907700389623642, Final Batch Loss: 0.01378594059497118
Epoch 249, Loss: 0.010731697548180819, Final Batch Loss: 0.0018435087986290455
Epoch 250, Loss: 0.005510726943612099, Final Batch Loss: 0.004589650314301252
Epoch 251, Loss: 0.00540820905007422, Final Batch Loss: 0.0026623986195772886
Epoch 252, Loss: 0.017216285690665245, Final Batch Loss: 0.010543699376285076
Epoch 253, Loss: 0.0055260497611016035, Final Batch Loss: 0.002551630837842822
Epoch 254, Loss: 0.007724222959950566, Final Batch Loss: 0.0036615219432860613
Epoch 255, Loss: 0.014949595090001822, Final Batch Loss: 0.011252754367887974
Epoch 256, Loss: 0.012371520511806011, Final Batch Loss: 0.007418765686452389
Epoch 257, Loss: 0.011971579864621162, Final Batch Loss: 0.009612812660634518
Epoch 258, Loss: 0.014450350776314735, Final Batch Loss: 0.009931380860507488
Epoch 259, Loss: 0.011036132229492068, Final Batch Loss: 0.0026

Epoch 368, Loss: 0.0090628566686064, Final Batch Loss: 0.00610940670594573
Epoch 369, Loss: 0.014194258983479813, Final Batch Loss: 0.00030015179072506726
Epoch 370, Loss: 0.004620229417923838, Final Batch Loss: 0.0039243074133992195
Epoch 371, Loss: 0.0022984453244134784, Final Batch Loss: 0.0006660163635388017
Epoch 372, Loss: 0.0021418322576209903, Final Batch Loss: 0.0015996844740584493
Epoch 373, Loss: 0.0010510481370147318, Final Batch Loss: 0.0003076922439504415
Epoch 374, Loss: 0.0012736371718347073, Final Batch Loss: 0.00027463282458484173
Epoch 375, Loss: 0.0021182170312386006, Final Batch Loss: 0.000471182371256873
Epoch 376, Loss: 0.009894606948364526, Final Batch Loss: 0.009242072701454163
Epoch 377, Loss: 0.0027782561955973506, Final Batch Loss: 0.0013830942334607244
Epoch 378, Loss: 0.003119909204542637, Final Batch Loss: 0.0013334908289834857
Epoch 379, Loss: 0.0017903833650052547, Final Batch Loss: 0.0005682306364178658
Epoch 380, Loss: 0.0034040441969409585, Final Bat

Epoch 495, Loss: 0.0008464625861961395, Final Batch Loss: 0.00032817121245898306
Epoch 496, Loss: 0.0029989758622832596, Final Batch Loss: 0.0005685841315425932
Epoch 497, Loss: 0.0038534447085112333, Final Batch Loss: 0.003055605571717024
Epoch 498, Loss: 0.0017882663232740015, Final Batch Loss: 0.0014400782529264688
Epoch 499, Loss: 0.0033123934408649802, Final Batch Loss: 0.0015655466122552752
Epoch 500, Loss: 0.000360765028744936, Final Batch Loss: 0.00018986911163665354
Epoch 501, Loss: 0.0017085261642932892, Final Batch Loss: 0.0010123519459739327
Epoch 502, Loss: 0.0008108469191938639, Final Batch Loss: 0.0005396079504862428
Epoch 503, Loss: 0.0013540777727030218, Final Batch Loss: 0.0009278235374949872
Epoch 504, Loss: 0.0012362125198706053, Final Batch Loss: 0.00010125288827111945
Epoch 505, Loss: 0.002298672392498702, Final Batch Loss: 0.001910950755700469
Epoch 506, Loss: 0.0035857873735949397, Final Batch Loss: 0.002165322657674551
Epoch 507, Loss: 0.0024035390233621, Final

...

Epoch 4506, Loss: 4.717498944728504e-05, Final Batch Loss: 4.689890192821622e-05
Epoch 4507, Loss: 1.3617736840387806e-05, Final Batch Loss: 4.378496669232845e-06
Epoch 4508, Loss: 1.1241129016070772e-05, Final Batch Loss: 8.997067766358668e-07
Epoch 4509, Loss: 1.0880756917686085e-05, Final Batch Loss: 9.765974937181454e-06
Epoch 4510, Loss: 7.176676808740012e-05, Final Batch Loss: 3.5196928365621716e-05
Epoch 4511, Loss: 2.026099673457793e-05, Final Batch Loss: 1.7394077076460235e-05
Epoch 4512, Loss: 2.7175931336387293e-06, Final Batch Loss: 1.1293831221337314e-06
Epoch 4513, Loss: 8.366671181647689e-06, Final Batch Loss: 3.1443462376046227e-06
Epoch 4514, Loss: 3.86021993108443e-05, Final Batch Loss: 3.8174030123627745e-06
Epoch 4515, Loss: 1.8107463120031753e-06, Final Batch Loss: 2.697419176911353e-07
Epoch 4516, Loss: 8.418202241955441e-07, Final Batch Loss: 5.142375698596879e-07
Epoch 4517, Loss: 3.04585017829595e-06, Final Batch Loss: 6.421439024961728e-07
Epoch 4518, Loss: 0.

Epoch 4616, Loss: 9.067487553693354e-06, Final Batch Loss: 1.8071696104016155e-06
Epoch 4617, Loss: 2.931441667897161e-05, Final Batch Loss: 2.5452869522268884e-05
Epoch 4618, Loss: 0.00019362299576641817, Final Batch Loss: 1.2111988780816318e-06
Epoch 4619, Loss: 0.00013648059393744916, Final Batch Loss: 5.9316123952157795e-05
Epoch 4620, Loss: 9.011189081320481e-05, Final Batch Loss: 2.242231630589231e-06
Epoch 4621, Loss: 1.9678856233440456e-06, Final Batch Loss: 1.3313126601133263e-07
Epoch 4622, Loss: 7.715455649304204e-05, Final Batch Loss: 2.4618984753033146e-05
Epoch 4623, Loss: 0.004261390576175472, Final Batch Loss: 0.004251847509294748
Epoch 4624, Loss: 3.7557871905846696e-06, Final Batch Loss: 6.282297704274242e-07
Epoch 4625, Loss: 4.462761444301577e-05, Final Batch Loss: 4.215421540720854e-06
Epoch 4626, Loss: 4.0435642176817055e-05, Final Batch Loss: 3.7959645851515234e-05
Epoch 4627, Loss: 9.529590670354082e-06, Final Batch Loss: 6.110617050580913e-06
Epoch 4628, Loss: 

Epoch 4721, Loss: 6.339572792057879e-06, Final Batch Loss: 8.909919415600598e-07
Epoch 4722, Loss: 9.747085130129562e-07, Final Batch Loss: 2.0274129042263667e-07
Epoch 4723, Loss: 0.00029352273122640327, Final Batch Loss: 0.00023167351901065558
Epoch 4724, Loss: 5.447817500225938e-06, Final Batch Loss: 4.68128007469204e-07
Epoch 4725, Loss: 6.61268506974011e-06, Final Batch Loss: 7.21336732567579e-07
Epoch 4726, Loss: 2.0729952893816517e-05, Final Batch Loss: 1.8270295186084695e-05
Epoch 4727, Loss: 2.876127229001213e-05, Final Batch Loss: 2.3145668137658504e-07
Epoch 4728, Loss: 0.0025785751022340264, Final Batch Loss: 7.679624104639515e-06
Epoch 4729, Loss: 0.0018202631499661948, Final Batch Loss: 0.00181566271930933
Epoch 4730, Loss: 1.3311390432590997e-05, Final Batch Loss: 1.2918484571855515e-05
Epoch 4731, Loss: 7.156123865570407e-05, Final Batch Loss: 5.434520426206291e-05
Epoch 4732, Loss: 1.2129178230679827e-05, Final Batch Loss: 6.1911705415695906e-06
Epoch 4733, Loss: 0.000

Epoch 4827, Loss: 6.870287506899331e-06, Final Batch Loss: 2.1143464437045623e-06
Epoch 4828, Loss: 1.0695370747271227e-05, Final Batch Loss: 3.2510151868336834e-06
Epoch 4829, Loss: 7.194841828095377e-06, Final Batch Loss: 6.172091161715798e-06
Epoch 4830, Loss: 6.890622898936272e-06, Final Batch Loss: 2.283940830238862e-06
Epoch 4831, Loss: 5.294259025845349e-06, Final Batch Loss: 1.8011802183082182e-07
Epoch 4832, Loss: 1.5519905858241145e-06, Final Batch Loss: 1.0441653586212851e-07
Epoch 4833, Loss: 8.398657655561692e-06, Final Batch Loss: 6.013851816533133e-06
Epoch 4834, Loss: 3.4697711726039415e-06, Final Batch Loss: 2.614395725686336e-06
Epoch 4835, Loss: 4.143973319514771e-05, Final Batch Loss: 4.729543888970511e-06
Epoch 4836, Loss: 5.9567106518443325e-06, Final Batch Loss: 4.086806256964337e-06
Epoch 4837, Loss: 1.2833447954108124e-05, Final Batch Loss: 1.0831021427293308e-05
Epoch 4838, Loss: 5.127906632651502e-07, Final Batch Loss: 1.2704033736099518e-07
Epoch 4839, Loss:

Epoch 4928, Loss: 8.640302922913179e-06, Final Batch Loss: 2.4972953838187095e-07
Epoch 4929, Loss: 6.691424687232939e-06, Final Batch Loss: 4.902570253761951e-06
Epoch 4930, Loss: 4.6835671625444775e-06, Final Batch Loss: 6.177975109267209e-08
Epoch 4931, Loss: 0.0034136007742517904, Final Batch Loss: 0.00341215031221509
Epoch 4932, Loss: 0.00024402654116784106, Final Batch Loss: 0.00024293512979056686
Epoch 4933, Loss: 0.0003179542660518564, Final Batch Loss: 0.0003168080002069473
Epoch 4934, Loss: 5.466195523240458e-05, Final Batch Loss: 5.41469817108009e-05
Epoch 4935, Loss: 7.307430746550381e-05, Final Batch Loss: 7.2357666795142e-05
Epoch 4936, Loss: 5.230060082794807e-06, Final Batch Loss: 1.5792464864716749e-06
Epoch 4937, Loss: 6.769624576463684e-06, Final Batch Loss: 7.970351703079359e-07
Epoch 4938, Loss: 2.673593326107948e-05, Final Batch Loss: 2.209820013376884e-05
Epoch 4939, Loss: 0.00011257342112003244, Final Batch Loss: 0.00011143047595396638
Epoch 4940, Loss: 5.117137

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[33  0  0]
 [ 0 24  0]
 [ 0  0 40]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        33
           1    1.00000   1.00000   1.00000        24
           2    1.00000   1.00000   1.00000        40

    accuracy                        1.00000        97
   macro avg    1.00000   1.00000   1.00000        97
weighted avg    1.00000   1.00000   1.00000        97



In [11]:
gen = Generator(z_dim = 106)
load_model(gen, "cGAN_UCI_Group_3_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=106, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=46, bias=True)
    (4): Tanh()
  )
)

In [12]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 3)

to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()

In [13]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(act_vectors[0], preds.cpu()))
print(metrics.classification_report(act_vectors[0], preds.cpu(), digits = 5))

[[30  0  0]
 [ 0 25  0]
 [ 0  0 42]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        30
           1    1.00000   1.00000   1.00000        25
           2    1.00000   1.00000   1.00000        42

    accuracy                        1.00000        97
   macro avg    1.00000   1.00000   1.00000        97
weighted avg    1.00000   1.00000   1.00000        97



# User Classifier

In [14]:
activities = [1, 3, 4]
users = [14, 15, 17]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [15]:
for k in range(len(y)):
    if y[k] == 14:
        y[k] = 0
    elif y[k] == 15:
        y[k] = 1
    else:
        y[k] = 2

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [17]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 2.217699885368347, Final Batch Loss: 1.119359016418457
Epoch 2, Loss: 2.2090342044830322, Final Batch Loss: 1.1033437252044678
Epoch 3, Loss: 2.212479829788208, Final Batch Loss: 1.1099441051483154
Epoch 4, Loss: 2.2085607051849365, Final Batch Loss: 1.1047834157943726
Epoch 5, Loss: 2.2087886333465576, Final Batch Loss: 1.104820966720581
Epoch 6, Loss: 2.2152249813079834, Final Batch Loss: 1.122309923171997
Epoch 7, Loss: 2.2055141925811768, Final Batch Loss: 1.1034730672836304
Epoch 8, Loss: 2.203434109687805, Final Batch Loss: 1.0988510847091675
Epoch 9, Loss: 2.198675274848938, Final Batch Loss: 1.0893535614013672
Epoch 10, Loss: 2.2043462991714478, Final Batch Loss: 1.105796456336975
Epoch 11, Loss: 2.204877734184265, Final Batch Loss: 1.1073110103607178
Epoch 12, Loss: 2.199535608291626, Final Batch Loss: 1.098240852355957
Epoch 13, Loss: 2.196805000305176, Final Batch Loss: 1.0940277576446533
Epoch 14, Loss: 2.2031521797180176, Final Batch Loss: 1.10940420627594
E

Epoch 122, Loss: 0.9032156765460968, Final Batch Loss: 0.4729790687561035
Epoch 123, Loss: 0.9008489549160004, Final Batch Loss: 0.41604024171829224
Epoch 124, Loss: 0.9041068851947784, Final Batch Loss: 0.4601951837539673
Epoch 125, Loss: 0.8790995180606842, Final Batch Loss: 0.44201284646987915
Epoch 126, Loss: 0.8469703793525696, Final Batch Loss: 0.40832793712615967
Epoch 127, Loss: 0.867369145154953, Final Batch Loss: 0.4666025936603546
Epoch 128, Loss: 0.8274988830089569, Final Batch Loss: 0.3870810568332672
Epoch 129, Loss: 0.8835499286651611, Final Batch Loss: 0.47291380167007446
Epoch 130, Loss: 0.840597003698349, Final Batch Loss: 0.3843845725059509
Epoch 131, Loss: 0.8582369089126587, Final Batch Loss: 0.43113043904304504
Epoch 132, Loss: 0.8290387392044067, Final Batch Loss: 0.41628366708755493
Epoch 133, Loss: 0.8293172121047974, Final Batch Loss: 0.43426942825317383
Epoch 134, Loss: 0.8132847845554352, Final Batch Loss: 0.3944415748119354
Epoch 135, Loss: 0.75202471017837

Epoch 246, Loss: 0.5520505309104919, Final Batch Loss: 0.31134089827537537
Epoch 247, Loss: 0.5333132445812225, Final Batch Loss: 0.30241477489471436
Epoch 248, Loss: 0.5306800901889801, Final Batch Loss: 0.24132665991783142
Epoch 249, Loss: 0.5490382015705109, Final Batch Loss: 0.2880197763442993
Epoch 250, Loss: 0.5449821352958679, Final Batch Loss: 0.3231327533721924
Epoch 251, Loss: 0.5074972361326218, Final Batch Loss: 0.24138103425502777
Epoch 252, Loss: 0.5275499373674393, Final Batch Loss: 0.27882882952690125
Epoch 253, Loss: 0.5859870612621307, Final Batch Loss: 0.341363787651062
Epoch 254, Loss: 0.5586755573749542, Final Batch Loss: 0.29169997572898865
Epoch 255, Loss: 0.5602753758430481, Final Batch Loss: 0.31673043966293335
Epoch 256, Loss: 0.5488959848880768, Final Batch Loss: 0.270349383354187
Epoch 257, Loss: 0.532558798789978, Final Batch Loss: 0.24442386627197266
Epoch 258, Loss: 0.5700439363718033, Final Batch Loss: 0.32090499997138977
Epoch 259, Loss: 0.5262443125247

Epoch 367, Loss: 0.48663952946662903, Final Batch Loss: 0.25856608152389526
Epoch 368, Loss: 0.490381196141243, Final Batch Loss: 0.2258189469575882
Epoch 369, Loss: 0.5069656819105148, Final Batch Loss: 0.2900691330432892
Epoch 370, Loss: 0.4928916245698929, Final Batch Loss: 0.25265732407569885
Epoch 371, Loss: 0.4926377832889557, Final Batch Loss: 0.2611995041370392
Epoch 372, Loss: 0.5429114699363708, Final Batch Loss: 0.33559301495552063
Epoch 373, Loss: 0.48660077154636383, Final Batch Loss: 0.2480163872241974
Epoch 374, Loss: 0.4822608232498169, Final Batch Loss: 0.21832561492919922
Epoch 375, Loss: 0.5053221732378006, Final Batch Loss: 0.26189085841178894
Epoch 376, Loss: 0.5449854731559753, Final Batch Loss: 0.27497291564941406
Epoch 377, Loss: 0.518206387758255, Final Batch Loss: 0.263232558965683
Epoch 378, Loss: 0.5273492932319641, Final Batch Loss: 0.31308987736701965
Epoch 379, Loss: 0.5006712228059769, Final Batch Loss: 0.2545808255672455
Epoch 380, Loss: 0.4743013978004

Epoch 494, Loss: 0.3706824332475662, Final Batch Loss: 0.18981927633285522
Epoch 495, Loss: 0.3963233679533005, Final Batch Loss: 0.20307476818561554
Epoch 496, Loss: 0.37290674448013306, Final Batch Loss: 0.20622912049293518
Epoch 497, Loss: 0.36939921975135803, Final Batch Loss: 0.16852693259716034
Epoch 498, Loss: 0.4657554626464844, Final Batch Loss: 0.2956545054912567
Epoch 499, Loss: 0.4020984023809433, Final Batch Loss: 0.19155602157115936
Epoch 500, Loss: 0.3644197881221771, Final Batch Loss: 0.18413835763931274
Epoch 501, Loss: 0.3889175355434418, Final Batch Loss: 0.21248821914196014
Epoch 502, Loss: 0.38314560055732727, Final Batch Loss: 0.20006249845027924
Epoch 503, Loss: 0.3846203237771988, Final Batch Loss: 0.177102193236351
Epoch 504, Loss: 0.39729154109954834, Final Batch Loss: 0.1759689897298813
Epoch 505, Loss: 0.42056845128536224, Final Batch Loss: 0.23509497940540314
Epoch 506, Loss: 0.38759854435920715, Final Batch Loss: 0.19140522181987762
Epoch 507, Loss: 0.4034

...

Epoch 4479, Loss: 0.07024407759308815, Final Batch Loss: 0.03282984718680382
Epoch 4480, Loss: 0.0341351144015789, Final Batch Loss: 0.01749284379184246
Epoch 4481, Loss: 0.07413435913622379, Final Batch Loss: 0.030774084851145744
Epoch 4482, Loss: 0.044535474851727486, Final Batch Loss: 0.022025562822818756
Epoch 4483, Loss: 0.028941906057298183, Final Batch Loss: 0.02170017547905445
Epoch 4484, Loss: 0.04879453778266907, Final Batch Loss: 0.0296846441924572
Epoch 4485, Loss: 0.045857563614845276, Final Batch Loss: 0.024805715307593346
Epoch 4486, Loss: 0.07409043191000819, Final Batch Loss: 0.0693596675992012
Epoch 4487, Loss: 0.03908157255500555, Final Batch Loss: 0.005648202262818813
Epoch 4488, Loss: 0.09913798049092293, Final Batch Loss: 0.06217543035745621
Epoch 4489, Loss: 0.0766170471906662, Final Batch Loss: 0.04135245829820633
Epoch 4490, Loss: 0.05101974681019783, Final Batch Loss: 0.02706022746860981
Epoch 4491, Loss: 0.04120447160676122, Final Batch Loss: 0.00584444450214

Epoch 4608, Loss: 0.020187570713460445, Final Batch Loss: 0.011394789442420006
Epoch 4609, Loss: 0.031565879471600056, Final Batch Loss: 0.016894929111003876
Epoch 4610, Loss: 0.0663855979219079, Final Batch Loss: 0.00815438199788332
Epoch 4611, Loss: 0.032147957012057304, Final Batch Loss: 0.016540560871362686
Epoch 4612, Loss: 0.032811093144118786, Final Batch Loss: 0.014391676522791386
Epoch 4613, Loss: 0.05054265446960926, Final Batch Loss: 0.02928652986884117
Epoch 4614, Loss: 0.023880979977548122, Final Batch Loss: 0.010287489742040634
Epoch 4615, Loss: 0.03720728540793061, Final Batch Loss: 0.007476492319256067
Epoch 4616, Loss: 0.052658991888165474, Final Batch Loss: 0.028512049466371536
Epoch 4617, Loss: 0.0308632031083107, Final Batch Loss: 0.02011345513164997
Epoch 4618, Loss: 0.035818912088871, Final Batch Loss: 0.025684967637062073
Epoch 4619, Loss: 0.0740491934120655, Final Batch Loss: 0.055537257343530655
Epoch 4620, Loss: 0.035441553220152855, Final Batch Loss: 0.025599

Epoch 4732, Loss: 0.0449655968695879, Final Batch Loss: 0.032367706298828125
Epoch 4733, Loss: 0.027405750937759876, Final Batch Loss: 0.005770317278802395
Epoch 4734, Loss: 0.07708261162042618, Final Batch Loss: 0.03589228168129921
Epoch 4735, Loss: 0.05157384555786848, Final Batch Loss: 0.038898371160030365
Epoch 4736, Loss: 0.04301067441701889, Final Batch Loss: 0.013137996196746826
Epoch 4737, Loss: 0.018483224557712674, Final Batch Loss: 0.015226674266159534
Epoch 4738, Loss: 0.046500956639647484, Final Batch Loss: 0.028619468212127686
Epoch 4739, Loss: 0.08420176804065704, Final Batch Loss: 0.06864803284406662
Epoch 4740, Loss: 0.04545214772224426, Final Batch Loss: 0.016817988827824593
Epoch 4741, Loss: 0.03709980100393295, Final Batch Loss: 0.01866946369409561
Epoch 4742, Loss: 0.028152827639132738, Final Batch Loss: 0.0028254021890461445
Epoch 4743, Loss: 0.07810785993933678, Final Batch Loss: 0.055673304945230484
Epoch 4744, Loss: 0.04813570901751518, Final Batch Loss: 0.0086

Epoch 4837, Loss: 0.043437860207632184, Final Batch Loss: 0.0031878382433205843
Epoch 4838, Loss: 0.05822737701237202, Final Batch Loss: 0.04547152668237686
Epoch 4839, Loss: 0.020229455083608627, Final Batch Loss: 0.01132376678287983
Epoch 4840, Loss: 0.0754506029188633, Final Batch Loss: 0.0395735539495945
Epoch 4841, Loss: 0.01745130540803075, Final Batch Loss: 0.005293041933327913
Epoch 4842, Loss: 0.03281147126108408, Final Batch Loss: 0.00928403902798891
Epoch 4843, Loss: 0.08422750979661942, Final Batch Loss: 0.01840360462665558
Epoch 4844, Loss: 0.02663939632475376, Final Batch Loss: 0.018537990748882294
Epoch 4845, Loss: 0.03993136901408434, Final Batch Loss: 0.015347021631896496
Epoch 4846, Loss: 0.05224723555147648, Final Batch Loss: 0.032569848001003265
Epoch 4847, Loss: 0.021772019565105438, Final Batch Loss: 0.01230320893228054
Epoch 4848, Loss: 0.025259443558752537, Final Batch Loss: 0.015621691942214966
Epoch 4849, Loss: 0.06366687454283237, Final Batch Loss: 0.01915859

Epoch 4944, Loss: 0.0345479529350996, Final Batch Loss: 0.01082288846373558
Epoch 4945, Loss: 0.10074398666620255, Final Batch Loss: 0.06399708986282349
Epoch 4946, Loss: 0.04414632171392441, Final Batch Loss: 0.012124080210924149
Epoch 4947, Loss: 0.05563867464661598, Final Batch Loss: 0.028673820197582245
Epoch 4948, Loss: 0.06465079262852669, Final Batch Loss: 0.04080589488148689
Epoch 4949, Loss: 0.04881568718701601, Final Batch Loss: 0.013188404031097889
Epoch 4950, Loss: 0.027285161428153515, Final Batch Loss: 0.008549981750547886
Epoch 4951, Loss: 0.01705992128700018, Final Batch Loss: 0.004262172617018223
Epoch 4952, Loss: 0.03276061359792948, Final Batch Loss: 0.008044279180467129
Epoch 4953, Loss: 0.0548320934176445, Final Batch Loss: 0.017160508781671524
Epoch 4954, Loss: 0.05021301284432411, Final Batch Loss: 0.01752842590212822
Epoch 4955, Loss: 0.030647968873381615, Final Batch Loss: 0.021949527785182
Epoch 4956, Loss: 0.06142880953848362, Final Batch Loss: 0.039106991142

In [18]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[27  1  1]
 [ 0 31  1]
 [ 0  0 36]]
              precision    recall  f1-score   support

           0    1.00000   0.93103   0.96429        29
           1    0.96875   0.96875   0.96875        32
           2    0.94737   1.00000   0.97297        36

    accuracy                        0.96907        97
   macro avg    0.97204   0.96659   0.96867        97
weighted avg    0.97016   0.96907   0.96898        97



In [19]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(usr_vectors[0], preds.cpu()))
print(metrics.classification_report(usr_vectors[0], preds.cpu(), digits = 5))

[[37  0  0]
 [ 0 28  0]
 [ 0  1 31]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        37
           1    0.96552   1.00000   0.98246        28
           2    1.00000   0.96875   0.98413        32

    accuracy                        0.98969        97
   macro avg    0.98851   0.98958   0.98886        97
weighted avg    0.99005   0.98969   0.98970        97

